L'objectif de ce TP est de programmer et de tester 2 algorithmes de classification très simples mais très efficaces. L'algorithme du Plus Proche Voisin (PPV) et le classifieur Baysien Naïf (CBN).

In [3]:
# Bibliothèques utiles
from sklearn import *
import numpy as np
from sklearn.neighbors import KNeighborsClassifier #algo des K plus proche voisins de sklearn
from sklearn import metrics #permet de calculer les distances entre données
from sklearn.naive_bayes import GaussianNB

## A. Plus proche voisin

L'algorithme du Plus Proche Voisin est un algorithme très simple de classification qui repose sur le principe suivant : **La classe de chaque donnée de test (à classer) doit être la classe de la donnée la plus proche (le plus similaire) parmi les données d'apprentissage**  

### 1) Création d'une fonction PPV(X, Y)

Cette fonction prend en entrée les données X et les étiquettes Y puis renvoie une étiquette pour chaque donnée prédite à partir du plus proche voisin de cette donnée.

In [4]:
def PPV(data, target):
    Y_pred=[]
    #calcule la distance euclidienne entre les données pour trouver celui qui est le plus proche d'une donnée quelconque
    distance = metrics.pairwise.euclidean_distances(data) 
    #Parcours de toutes les distances
    for i, point in enumerate(distance): #i = clé ; point = valeur ; enumerate(distance) = un dictionnaire
        # On supprime la valeur en position i de la matrice point courante par delete(point,i)
        #puis on les stocke dans une matrice contenant les voisins de l'elt à la position i de point
        #la matrice point passe de 150 observations à 149
        voisins = np.delete(point,i)
        # par la suite on récupère l'indice du minimum de la nouvelle matrice par argmin
        #Le plus petit voisin de l'elt à la position i de la matrice point
        # il correspond à la distance minimale et donc à l'indice de l'elt le plus proche de i de point
        min_voisin = np.argmin(voisins)
        #on récupère l'étiquette du plus proche voisin de l'elt à la position i       
        Y_pred.append(target[min_voisin])
    return np.array(Y_pred)

### 2) Erreur de prédiction : Pourcentage d'étiquettes mal prédites

In [5]:
def PPV_erreur(data, target):
    Y_pred=[]
    erreur = []
    distance = metrics.pairwise.euclidean_distances(data) 
    #Parcours de toutes les distances
    for i, point in enumerate(distance): 
        voisins = np.delete(point,i)
        min_voisin = np.argmin(voisins)
        Y_pred.append(target[min_voisin])
        #Y_pred = np.array(Y_pred)
    
    #calcul de l'erreur
    erreur.append(np.mean(Y_pred != Y))

    return (erreur[0]*100)

### 3) Test sur les données Iris

In [6]:
# Données iris
iris = datasets.load_iris()
X = iris.data #les données
Y = iris.target # les étiquettes

# étiquettes des données prédites
Y_pred = PPV(X,Y)
print("Etiquettes des données prédites : ", Y_pred)

# Erreur de prédiction
erreur = PPV_erreur(X,Y)
print("Erreur de prédiction : ",erreur,"%")

Etiquettes des données prédites :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Erreur de prédiction :  4.0 %


### 4) Test de la fonction des k plus proches voisins de sklearn (avec k=1)

In [7]:
# Test avec k=1
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, Y) 
#prédiction des étiquettes
pred = knn.predict(X)
error = []
error.append(np.mean(pred != Y))
print("Erreur de prédiction pour k=1 : ", error[0]*100, "%")

Erreur de prédiction pour k=1 :  0.0 %


In [8]:
# Test avec d'autres valeurs
# On considère les k de 1 à 10
for k in range(1,11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, Y) 
    #prédiction des étiquettes
    pred = knn.predict(X)
    error = []
    error.append(np.mean(pred != Y))
    print("Erreur de prédiction pour k = ",k," : ",round(error[0]*100, 2), "%")

Erreur de prédiction pour k =  1  :  0.0 %
Erreur de prédiction pour k =  2  :  2.0 %
Erreur de prédiction pour k =  3  :  4.0 %
Erreur de prédiction pour k =  4  :  4.0 %
Erreur de prédiction pour k =  5  :  3.33 %
Erreur de prédiction pour k =  6  :  2.67 %
Erreur de prédiction pour k =  7  :  2.67 %
Erreur de prédiction pour k =  8  :  2.0 %
Erreur de prédiction pour k =  9  :  2.0 %
Erreur de prédiction pour k =  10  :  2.0 %


Les résultats sont différents de ceux obtenus avec notre algorithme car il n'y a pas d'erreur de prédiction

## B. Classifieur Bayesien Naïf

L'algorithme du classifieur Bayésien Naïf est un algorithme de classification basé sur le calcul de probabilité d'appartenance à chaque classe. C'est à dire que la donnée de test (à classer) sera affectée à la classe la plus probable.

### 1. Fonction CBN(X,Y)

In [9]:
def CBN(X, Y):
    # liste des moyennes (barycentre) de la classe
    moy = [np.mean(X[(Y==0)], axis=0), np.mean(X[(Y==1)], axis=0), np.mean(X[(Y==2)], axis=0)]
    # on convertit cette liste de moyenne en tableau numpy plus facile à manipuler
    moy=np.asarray(moy)

    # distance euclidienne entre une donnée X et le barycentre (moyenne) de la classe
    dxk = metrics.pairwise.euclidean_distances (X, moy)
    
    # somme des distances entre la donnée X et chaque barycentre de chaque classe
    dxb=np.sum(dxk, axis=1)
    
    pxk = np.zeros(450,dtype=float).reshape((150, 3))
    for i in range (0,len(dxb)):
        for j in range (0, 3):
            # probabilité qu'une donnée X ait la valeur Xi pour la variable i connaissant sa classe
            pxk[i][j]=1-(dxk[i][j]/dxb[i]) 
    
    # propabilité d'appartenance à chaque classe
    Ybay=np.argmax(pxk, axis=1)
    return Ybay    

### 2. Erreur de prédiction (pourcentage d'étiquettes mal prédites)

In [10]:
def CBN_erreur(X, Y):
    #Y_pred=[]
    erreur = []
    # liste des moyennes (barycentre) de la classe
    moy = [np.mean(X[(Y==0)], axis=0), np.mean(X[(Y==1)], axis=0), np.mean(X[(Y==2)], axis=0)]
    # on convertit cette liste de moyenne en tableau numpy plus facile à manipuler
    moy=np.asarray(moy)

    # distance euclidienne entre une donnée X et le barycentre (moyenne) de la classe
    dxk = metrics.pairwise.euclidean_distances (X, moy)
    
    # somme des distances entre la donnée X et chaque barycentre de chaque classe
    dxb=np.sum(dxk, axis=1)
    
    # On crée une matrice qui doit contenir les probabilités de chaque puis on le redimensionne
    # en 2 D. Sachant qu'ici on a 3 classe unique et comme pour chaque classe
    # il y aura 150 éléments, on aura donc en tout 450 éléments dans cette matrice
    pxk = np.zeros(450,dtype=float).reshape((150, 3))
    for i in range (0,len(dxb)):
        for j in range (0, 3):
            # probabilité qu'une donnée X ait la valeur Xi pour la variable i connaissant sa classe
            pxk[i][j]=1-(dxk[i][j]/dxb[i]) 
    
    # propabilité d'appartenance à chaque classe
    Y_pred=np.argmax(pxk, axis=1)
    
    # calcul de l'erreur
    erreur.append(np.mean(Y_pred != Y))
    
    return round((erreur[0]*100), 2)

#### - Test sur les données iris

In [11]:
erreur = CBN_erreur(X, Y)
print("Le pourcentage d'étiquettes mal prédites est de ",erreur,"%")

Le pourcentage d'étiquettes mal prédites est de  7.33 %


### 3. Test de la fonction du classifieur Bayesien Naïf inclu dans sklearn

In [12]:
gnb = GaussianNB()
gnb.fit(X, Y)
y_pred = gnb.predict(X)
print(y_pred)
e = []
e.append(np.mean(y_pred != Y))
print("Erreur de prédiction : ", e[0]*100, "%")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Erreur de prédiction :  4.0 %


In [13]:
gnb

GaussianNB(priors=None, var_smoothing=1e-09)

Les résultats ont bel et bien différents car avec notre fonction on a eu une erreur de prédiction de 7 % mais avec la fonction du classifieur bayesien naif inclu dans sklearn, l'erreur est de 4%